In [1]:
#@title Download data from GCP bucket
import sys

if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

Copying gs://indaba-data/README.txt...
Copying gs://indaba-data/train/train.csv...
Copying gs://indaba-data/train/train_mut.pt...
Copying gs://indaba-data/test/test.csv...
Copying gs://indaba-data/test/test_mut.pt...
Copying gs://indaba-data/train/train_wt.pt...
Copying gs://indaba-data/test/test_wt.pt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").



In [2]:
#@title Imports and moving to working directory
import torch 
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader


# move to data folder
%cd indaba-data

/content/indaba-data


In [3]:
# Load Embedding tensors & Traing csv
# Embeddings were calculated using the ESM 650M pretrained model 
# Tensor shape of embedded data:  [data_len,1280] 
# There are no sequences in the Embedding tensors as we've performed an average of it (torch.mean(embed, dim=1))
# More details in https://huggingface.co/facebook/esm2_t33_650M_UR50D

wt_emb = torch.load("train/train_wt.pt")
mut_emb = torch.load("train/train_mut.pt")
df = pd.read_csv("train/train.csv")

In [4]:
len(df)

339778

In [5]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Split data into train and validation
wt_emb_train, wt_emb_val, mut_emb_train, mut_emb_val, df_train, df_val = train_test_split(wt_emb, mut_emb, df, test_size=0.21, random_state=42)

# Define the dataset class
class EmbeddingDataset(Dataset):
  def __init__(self, wt_pt, mut_pt, data_df):
    self.pt_wt = wt_pt
    self.pt_mut = mut_pt
    self.df = data_df

  def __len__(self):
    return len(self.pt_wt)

  def __getitem__(self, index):
    if "ddg" in self.df.columns:
      df_out = torch.Tensor([self.df.iloc[index]["ddg"]])
    else:
      df_out = torch.Tensor([self.df.iloc[index]["ID"]])

    return self.pt_wt[index,:], self.pt_mut[index,:], df_out

# Create separate datasets for the training and validation sets
train_dataset = EmbeddingDataset(wt_emb_train, mut_emb_train, df_train.reset_index(drop=True))
val_dataset = EmbeddingDataset(wt_emb_val, mut_emb_val, df_val.reset_index(drop=True))

# Create dataloaders for the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [6]:
len(train_dataloader)

8389

In [7]:
len(val_dataloader)


2230

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, wt_emb, mut_emb):
        x = torch.cat((wt_emb, mut_emb), dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define the dimensions
input_dim = wt_emb.shape[1]*2  # times 2 because we're concatenating the two embeddings
hidden_dim = 256
output_dim = 1  # ddG value

model = MLP(input_dim, hidden_dim, output_dim)


In [9]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 30  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    for wt_emb, mut_emb, ddg in train_dataloader:
        optimizer.zero_grad()
        outputs = model(wt_emb, mut_emb)
        loss = criterion(outputs, ddg)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    with torch.no_grad():
        val_losses = []
        for wt_emb, mut_emb, ddg in val_dataloader:
            outputs = model(wt_emb, mut_emb)
            loss = criterion(outputs, ddg)
            val_losses.append(loss.item())
    avg_val_loss = sum(val_losses) / len(val_losses)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')


Epoch 1, Train Loss: 0.9708378319661287, Val Loss: 0.9196932341352172
Epoch 2, Train Loss: 0.8911184600154411, Val Loss: 0.8455181624055443
Epoch 3, Train Loss: 0.8774642883480795, Val Loss: 0.8317769894284518
Epoch 4, Train Loss: 0.8674177270098883, Val Loss: 0.8100190595794686
Epoch 5, Train Loss: 0.8646743443284647, Val Loss: 0.810587113013182
Epoch 6, Train Loss: 0.8593442161015865, Val Loss: 0.8057269820851596
Epoch 7, Train Loss: 0.8563544313871956, Val Loss: 0.8976651036017679
Epoch 8, Train Loss: 0.8550477767320338, Val Loss: 0.8345197615468448
Epoch 9, Train Loss: 0.8491192520506718, Val Loss: 0.7962046114585859
Epoch 10, Train Loss: 0.8478792056979518, Val Loss: 0.8142912161189879
Epoch 11, Train Loss: 0.8429595853794544, Val Loss: 0.7884069915977828
Epoch 12, Train Loss: 0.8429929117314338, Val Loss: 0.7950324694337866
Epoch 13, Train Loss: 0.8396146297277206, Val Loss: 0.8474395713464027
Epoch 14, Train Loss: 0.8373200567313749, Val Loss: 0.7893206033738739
Epoch 15, Train 